# Урок 3. Embedding word2vec fasttext

для новостных статей задание такое же
1. на основе word2vec/fasttext слоя Embedding реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [ ]:
!pip install natasha stop_words annoy gensim pymorphy3 pandarallel ipywidgets --break-system-packages

In [2]:
!tar -xzvf gazeta_jsonl.tar.gz

gazeta_train.jsonl
gazeta_test.jsonl
gazeta_val.jsonl


In [ ]:
import numpy as np
import pandas as pd
from pandarallel import pandarallel

import string
import pickle
import annoy
import swifter

from pymorphy3 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

pandarallel.initialize(use_memory_fs = False)

## Объединяем данные в один датафрейм

In [4]:
df_train = pd.read_json(path_or_buf='gazeta_train.jsonl', lines=True)
df_val = pd.read_json(path_or_buf='gazeta_val.jsonl', lines=True)
df_test = pd.read_json(path_or_buf='gazeta_test.jsonl', lines=True)

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
df.head()

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63435 entries, 0 to 63434
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   url      63435 non-null  object        
 1   text     63435 non-null  object        
 2   title    63435 non-null  object        
 3   summary  63435 non-null  object        
 4   date     63435 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 2.4+ MB


In [28]:
text_lengths = df['text'].apply(len)
summary_lengths = df['summary'].apply(len)

print(f'Средняя длина text - {round(np.average(text_lengths))} ')
print(f'Средняя длина summary - {round(np.average(summary_lengths))} ')

Средняя длина text - 4519 
Средняя длина summary - 312 


## Предварительная подготовка текста

In [7]:
morpher = MorphAnalyzer()
stopwords = set(get_stop_words("ru"))
punkt = set(string.punctuation)

def preprocess_txt(line, morpher=morpher, sw=stopwords, punkt=punkt):
    spls = "".join(i for i in line.strip() if i not in punkt).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [8]:
df['preprocessed_summary'] = df['summary'].parallel_apply(preprocess_txt)
df.head()

,url,text,title,summary,date,preprocessed_summary
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39,"[2011, россия, уйти, 80, млрд, считать, минэко..."
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09,"[юлия, соловьёв, эксдиректор, холдинг, «профме..."
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14,"[суд, арестовать, месяц, четверо, эксчиновник,..."
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50,"[мнение, предприниматель, повод, вступление, в..."
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01,"[канал, завершиться, сезон, шоу, «голос», побе..."


In [10]:
window_size = 5
vector_size = 500

modelW2V = Word2Vec(sentences=df['preprocessed_summary'], 
                    vector_size=vector_size, 
                    window=window_size, 
                    min_count=1)

modelFT = FastText(sentences=df['preprocessed_summary'], 
                   vector_size=vector_size, 
                   window=window_size, 
                   min_count=1)

## Функция для получения ближайших статей

In [16]:
def get_response(question, index, model, index_map, vector_size, nn=5, limit=10):
    question = preprocess_txt(question)
    vector = np.zeros(vector_size)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, nn, )
    return [index_map[i] for i in answers][:limit]

In [11]:
w2v_index = annoy.AnnoyIndex(vector_size, 'angular')
ft_index = annoy.AnnoyIndex(vector_size, 'angular')

index_map = {}
counter = 0

for i in range(len(df['preprocessed_summary'])):
    n_w2v = 0
    n_ft = 0
    index_map[i] = df['text'][i]
    
    vector_w2v = np.zeros(vector_size)
    vector_ft = np.zeros(vector_size)
    for word in df['preprocessed_summary'][i]:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(i, vector_w2v)
    ft_index.add_item(i, vector_ft)


w2v_index.build(10)
ft_index.build(10)

True

## Поиск ближайших по смыслу статей

In [20]:
TEXT = "Экономика"
get_response(TEXT, w2v_index, modelW2V, index_map, vector_size, limit=1)[0][:100] + '...'

'Министерство экономического развития внесло в Минфин проект уточненного социально-экономического про...'

In [21]:
get_response(TEXT, ft_index, modelFT, index_map, vector_size, limit=1)[0][:100] + '...'

'«Сегодня фондовые индексы во многих странах вышли на докризисные уровни, но они не обеспечены такими...'

In [25]:
TEXT = "Спорт"
get_response(TEXT, w2v_index, modelW2V, index_map, vector_size, limit=1)[0][:100] + '...'

'Во вторник в Министерстве спорта, туризма и молодежной политики были подведены итоги выступления сбо...'

In [26]:
get_response(TEXT, ft_index, modelFT, index_map, vector_size, limit=1)[0][:100] + '...'

'Несколько участников чемпионата мира по водным видам спорта в южнокорейском Кванджу получили поврежд...'